In [1]:
cd ..

c:\Users\14102\Brown\Realizing_Rights\RealizingRights


In [2]:
from utilities.web_utils import recurse_scan_all_unique_links_in_site, closest_link_match, make_https, prepend_root_to_url, make_driver_utils
from url_scraper.link_data import LinkData
from main import main
from objects.scrape_lists import blacklist_terms, link_keywords, board_meeting_keywords, social_media_sites
from url_scraper.web_scraper import DistrictWebsiteScraper
from utilities.data_utils import format_input_district_df, format_output_boe_df

import os
import re
from concurrent.futures.thread import ThreadPoolExecutor
import queue
import pandas as pd

In [3]:
# source_info = {'path': 'data/USSchoolDistrictWebsiteInfo.xlsx', 'sheet': 'ELSI Export', 'head_row': 6}
# write_file_path = 'data/SampleOutput.csv'
    
# main(source_info=source_info, write_file_path=write_file_path, verbose=False, max_dist_runs=None, out_file_name=write_file_path)

In [32]:
current_output = pd.read_csv('data/SampleOutput.csv')

def check_malformed(url: str) -> bool:
    if len([char.start() for char in re.finditer('//', url)]) > 1:
        return True
    elif 'javascript' in url:
        return True
    else:
        return False

# current_output['Board Meeting URL Link'] = current_output.apply(lambda x: str(x['Board Meeting URL Link']), axis=1)
current_pop_output = current_output[~current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)
current_unpop_output = current_output[current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)

current_pop_output['BadBoardURL'] = current_pop_output.apply(lambda x: check_malformed(url=x['Board Meeting URL Link']), axis=1)
malformed_brd_mt_df = current_pop_output[current_pop_output['BadBoardURL']]
current_pop_output = current_pop_output[~current_pop_output['BadBoardURL']]
current_pop_output.drop('BadBoardURL', axis=1, inplace=True)
current_pop_output = current_pop_output.append(current_unpop_output)
malformed_brd_mt_df

,District ID,Board Meeting URL Text,Board Meeting URL Link,Board Meeting Link Depth,youtube,vimeo,facebook,twitter,Agency Name,State Name,State Abbr,Agency ID,NumSchools,LowGrade,HighGrade,NumStudents,URL,BadBoardURL
11986,4846710.0,Board,https://www.zcisd.org/apps//apps/pages/index.j...,0.0,NaN,NaN,0.0,NaN,ZAPATA COUNTY ISD,Texas,TX,4846710.0,6.0,Prekindergarten,12th Grade,3287.0,https://www.zcisd.org,True
12001,620910.0,Request Board Meeting Agendas,https://lcmschools.org//cms/module/selectsurve...,0.0,0.0,NaN,0.0,0.0,LARKSPUR-CORTE MADERA,California,CA,620910.0,3.0,Kindergarten,8th Grade,1354.0,https://www.lcmschools.org,True
12012,2613560.0,HERE,https://http://www.cmdhd.org,1.0,NaN,NaN,1.0,NaN,EVART PUBLIC SCHOOLS,Michigan,MI,2613560.0,3.0,Prekindergarten,12th Grade,874.0,https://www.evartps.org,True
12017,3905022.0,Board of Education Minutes,https://www.mcdonald.k12.oh.us//boardofeducati...,2.0,NaN,NaN,0.0,NaN,MCDONALD LOCAL,Ohio,OH,3905022.0,2.0,Kindergarten,12th Grade,736.0,https://www.mcdonald.k12.oh.us,True
12028,5000024.0,Maintenance Requests,https://accounts.zoho.com/signin?servicename=s...,0.0,NaN,NaN,NaN,NaN,RIVENDELL INTERSTATE SCHOOL DISTRICT,Vermont,VT,5000024.0,3.0,Prekindergarten,12th Grade,442.0,https://www.rivendellschool.org,True
12041,1100082.0,Get,https://www.google.com/maps/dir//richard+wrigh...,0.0,0.0,NaN,0.0,0.0,RICHARD WRIGHT PCS FOR JOURNALISM AND MEDIA ARTS,District of Columbia,DC,1100082.0,1.0,8th Grade,12th Grade,296.0,https://www.richardwrightpcs.org,True
12044,601978.0,here,https://waldorfanswers.org//index.htm,1.0,1.0,NaN,0.0,NaN,STONE BRIDGE DISTRICT,CALIFORNIA,CA,601978.0,1.0,Kindergarten,8th Grade,269.0,https://www.stonebridgeschool.org,True
12047,3417340.0,Board,https://accounts.google.com/accountchooser?con...,0.0,NaN,NaN,NaN,NaN,WENONAH BORO SCHOOL DISTRICT,New Jersey,NJ,3417340.0,1.0,Prekindergarten,6th Grade,172.0,https://www.wenonahsd.org,True
12050,400448.0,June 14 22 Board Strategic Planning Meeting Ag...,https://www.shellya7.sg-host.com///board-membe...,1.0,0.0,NaN,0.0,NaN,DESERT SKY COMMUNITY SCHOOL INC. (88308),Arizona,AZ,400448.0,1.0,Kindergarten,5th Grade,53.0,https://www.desertskyschool.org,True


In [4]:
current_output = pd.read_csv('data/SampleOutput.csv')
missing_brd_mt_df = current_output[current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)

In [25]:
pipeline = queue.Queue()
full_df = pd.DataFrame()
urls_processed = 0

def reprocess_missing_data(source: pd.DataFrame, write_file_path: str, verbose: bool, max_dist_runs: int) -> None:
    global full_df

    district_df_to_use = source[:max_dist_runs]
    # Scrape data
    
    url_list_to_process = list(district_df_to_use['URL'])
    num_urls = len(url_list_to_process)
    url_id_zip = [(url, a_id) for url, a_id in zip(list(district_df_to_use['URL']), list(district_df_to_use['District ID']))]

    with ThreadPoolExecutor(max_workers=12) as executor:

        _ = executor.submit(build_output_df, num_urls)
        executor.map(scrape_district, url_id_zip)

    full_df.to_csv(write_file_path, index=False)


def scrape_district(url_id_tuple: tuple):
    global urls_processed
    print(f'Parsing {url_id_tuple[0]}...')
    try:
        district_scraper = DistrictWebsiteScraper(url=url_id_tuple[0], agency_id=url_id_tuple[1], verbose=False)
        district_scraper.find_board_meeting_and_social_media_links()
        pipeline.put(district_scraper.url_data)
        print(f'{url_id_tuple[0]} Done')
    except Exception as e:
        print(f"{url_id_tuple[0]} Error: {e}")
        urls_processed += 1


def build_output_df(num_urls: int):
    global full_df
    global urls_processed
    while urls_processed < num_urls:
        url_info_dict = pipeline.get()
        url_info_row = format_output_boe_df(url_data=url_info_dict, social_media_sites=social_media_sites)
        full_df = full_df.append(url_info_row)
        urls_processed += 1
        print(urls_processed)


In [26]:
write_file_path = 'data/reprocessed.csv'

reprocess_missing_data(source=malformed_brd_mt_df, write_file_path=write_file_path, max_dist_runs=9999, verbose=False)


Parsing https://www.smasd.org...
Parsing https://www.pcsd1.org...
Parsing https://www.godfrey-lee.org...
Parsing https://www.casdk12.net...
Parsing https://www.nfdlschools.org...
Parsing https://www.jacksoncoschools.com...
Parsing https://www.maustonschools.org...
Parsing https://www.berlin.k12.wi.us...
Parsing https://www.brps.org...
Parsing https://www.mapsk12.org...
Parsing https://www.franklinisd.net...
https://www.nfdlschools.org Done1

Parsing https://www.moniteau.org...
https://www.mapsk12.org Done
2
Parsing https://www.closterschools.org...
https://www.jacksoncoschools.com Done
3
https://www.godfrey-lee.org Done
4
Parsing https://www.apolloridge.com...
Parsing https://www.pas.k12.mn.us...
https://www.smasd.org Done5

https://www.franklinisd.net Done
6
Parsing https://morleystanwood.org...
Parsing https://www.peshtigo.k12.wi.us...
https://www.berlin.k12.wi.us Done
7
https://www.moniteau.org Done8

Parsing https://www.westfield.k12.wi.us...
https://www.casdk12.net Done
9
Parsing 

In [27]:
source_info = {'path': 'data/USSchoolDistrictWebsiteInfo.xlsx', 'sheet': 'ELSI Export', 'head_row': 6}
xlsx_input = pd.read_excel(io=source_info['path'], sheet_name=source_info['sheet'], header=source_info['head_row'])
district_df = format_input_district_df(dist_df=xlsx_input)
pop_output = current_output[~current_output['Board Meeting URL Link'].isna()]


In [28]:
full_df_info = district_df.merge(full_df, left_on='Agency ID', right_on='District ID')

In [29]:
# updated_output_df = pop_output.append(full_df_info)
updated_output_df = current_pop_output.append(full_df_info)
updated_output_df

,District ID,Board Meeting URL Text,Board Meeting URL Link,Board Meeting Link Depth,youtube,vimeo,facebook,twitter,Agency Name,State Name,State Abbr,Agency ID,NumSchools,LowGrade,HighGrade,NumStudents,URL
0,622710.0,Board Meeting Agendas,https://boe.lausd.net/lausdbdmtgagendas,0.0,0.0,2.0,0.0,0.0,LOS ANGELES UNIFIED,California,CA,622710.0,778.0,Kindergarten,12th Grade,435958.0,https://www.lausd.net
1,1709930.0,Join Meeting,https://meet.google.com/riy-odxs-xmx,1.0,0.0,NaN,0.0,0.0,CITY OF CHICAGO SD 299,Illinois,IL,1709930.0,646.0,Prekindergarten,12th Grade,329836.0,https://www.cps.edu
2,1200180.0,Board Meetings & Workshops,https://www.browardschools.com/boardmeetings,0.0,0.0,NaN,0.0,0.0,BROWARD,Florida,FL,1200180.0,332.0,Prekindergarten,12th Grade,256037.0,https://www.browardschools.com
3,4823640.0,Board Meetings,https://www.houstonisd.org/domain/7947,0.0,0.0,NaN,0.0,0.0,HOUSTON ISD,Texas,TX,4823640.0,274.0,Prekindergarten,12th Grade,194607.0,https://www.houstonisd.org
4,1201500.0,Board Meetings,https://www.palmbeachschools.org/domain/255,0.0,0.0,NaN,0.0,0.0,PALM BEACH,Florida,FL,1201500.0,235.0,Prekindergarten,12th Grade,187943.0,https://www.palmbeachschools.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2626100.0,Board of Education,https://sjcisd.org/your-isd/board-of-education,0,None,None,None,None,NOTTAWA COMMUNITY SCHOOL,Michigan,MI,2626100.0,1.0,Prekindergarten,8th Grade,151.0,https://www.sjcisd.org
81,600002.0,NaN,NaN,NaN,0.0,None,0.0,0.0,CALIFORNIA SCHOOL FOR THE BLIND (STATE SPECIAL...,California,CA,600002.0,1.0,Kindergarten,12th Grade,62.0,https://www.csb-cde.ca.gov
82,631020.0,Board Meeting,https://www.pleasant-valley-school.org/apps/ev...,0,None,None,0,None,PLEASANT VALLEY JOINT UNION ELEMENTARY,California,CA,631020.0,1.0,Kindergarten,6th Grade,54.0,https://www.pleasant-valley-school.org
83,409540.0,NaN,NaN,NaN,NaN,None,NaN,NaN,YOUNG ELEMENTARY DISTRICT (4213),Arizona,AZ,409540.0,2.0,Prekindergarten,12th Grade,54.0,https://www.youngschool.org


In [31]:
updated_output_df.to_csv('data/SampleOutput.csv', index=False)

In [7]:
import re
re.findall(r'[^\.]+', 'www.hello.com')

['www', 'hello', 'com']